# This Code snippet below is based on the Real time Air Quality Index from various locations in INDIA

ORG is "Ministry of Environment, Forest and Climate Change", "Central Pollution Control Board"
sector is "Environment and Forest", "Vehicular Air Pollution","Residential Air Pollution", and "Industrial Air Pollution".

## This file consist of
01. Triggering the open API 
02. Extracting the Json Output to a CSV file in the local path 
The output file is competation.csv

Step01: installing supporting libraries

In [31]:
!pip3 install requests
!pip3 install jsonpath-ng


'pip3' is not recognized as an internal or external command,
operable program or batch file.
'pip3' is not recognized as an internal or external command,
operable program or batch file.


Function to verify whether the json object have inner array of data 

In [32]:
def getCsvKeylist(data, keys, list, parent):
    # Check if key has inner JSON
    for key in keys:
        if isinstance(data.get(key), dict):
            # Inner JSON found!
            childData = data.get(key)
            childKeys = childData.keys()
            if parent:
                key = parent + "." + key
            # Recursive call for inner json      
            getCsvKeylist(childData, childKeys, list, key)
            
        else:
            if parent: 
                key = parent + "." + key
            list.append(key)
        
    return list

This Function identifies the child data in an recurssive order and lists all the child tree structure in the Json 

In [33]:
def getCsvDataRow(data, keyData, keys, list):
    # We need to pass the keyData, to iterate over all the keys which may not present in the current data
    for key in keys:
        if isinstance(keyData.get(key), dict):
            # Inner JSON found for key data!
            childKeyData = keyData.get(key)
            childKeys = childKeyData.keys()

            # Inner json for actual data
            childData = ""
            if data:
                childData = data.get(key)

            # Recursive call for inner json      
            getCsvDataRow(childData, childKeyData, childKeys, list)
            
        else:
            if data:
                list.append(data.get(key))
            else:
                list.append("")     
        
    return list

In [34]:
from jsonpath_ng import parse


def getValueFromJsonPath(jsonData, jsonPath):
    jsonPathExpression = "$." + jsonPath
    expr = parse(jsonPathExpression)
    jsonValue = ""
    # Use the expression to get the value of the path from the JSON data
    match = expr.find(jsonData)
    if match:
        jsonValue = match[0].value

    return jsonValue 

This Functions hits the open API and verify whether the API is successful or not and check for the child "RECORDS" in the JSON   

In [35]:
import requests
import json

def getJsonPayloadFromApi():
   # headers = {'X-Auth-Token': '579b464db66ec23bdd0000017789d1ca79674d856cd0d050548fe535' }
    url = 'https://api.data.gov.in/resource/3b01bcb8-0b14-4abf-b6f2-c1bfd384ba69?api-key=579b464db66ec23bdd0000017789d1ca79674d856cd0d050548fe535&format=json&offset=0&filters[country]=India&limit=1000'

    # Send request to the API
    #response = requests.get(url, headers=headers)
    response = requests.get(url)
    
    # Check if the request was successful
    if response.ok:
        # Convert response to JSON and take competitions
        return json.loads(response.text).get("records")           

    else:
        print(f'Request failed with status code {response.status_code}')


In Case the API fails fetch the prepopulated data that is already existing in this path as the text file.

In [36]:
import json

def getJsonPayloadFromFile():

    # Provide correct json file path,
    filePath ='C:/Users/Manu/Downloads/StudiesNCI/MichealBrad_DB/projects/JsonPayload.txt'
    with open(filePath, 'r') as f:
        response = f.read()
    return json.loads(response).get("records")  

Once the data is fetched through API or a file write the data to competation.csv file in the location where this file is stored.
Each time the old competation.csv file is deleted and a new file with the data is created 

In [37]:
import json
import csv
import traceback
import sys


try:

    # Change method to read from API or file, and also update the corresponding methods with correct path or url
    competitionsData = getJsonPayloadFromApi()
    # competitionsData = getJsonPayloadFromFile()

    if competitionsData is not None:
        # Convert response to JSON and take competitions
        
        # Get CSV header columns
        # For getting key data, take a competition json from reponse having all the expected keys
        #keyData = '{"id":2022,"area":{"id":2016,"name":"Austria","code":"AUT","flag":"https:\/\/crests.football-data.org\/816.svg"},"name":"Playoffs 1\/2","code":"APL","type":"PLAYOFFS","emblem":null,"plan":"TIER_TWO","currentSeason":{"id":24,"startDate":"2018-05-31","endDate":"2018-06-03","currentMatchday":null,"winner":{"id":2022,"name":"spusu SKN St. P\u00f6lten","shortName":"St. P\u00f6lten","tla":"STP","crest":"https:\/\/crests.football-data.org\/2022.svg","address":"Bimbo Binder Promenade 9 Sankt P\u00f6lten 3100","website":"http:\/\/www.skn-stpoelten.at","founded":2000,"clubColors":"Yellow \/ Blue","venue":"NV Arena","lastUpdated":"2021-05-22T07:53:43Z"}},"numberOfAvailableSeasons":1,"lastUpdated":"2018-08-23T15:47:33Z"}'
        keyData= '{"id":"1","country": "India","state": "Andhra_Pradesh","city": "Amaravati","station": "Secretariat, Amaravati - APPCB","last_update": "09-04-2023 05:00:00","pollutant_id": "PM2.5","pollutant_min": "12","pollutant_max": "54","pollutant_avg": "30","pollutant_unit": "NA"}'
        keyDataJson = json.loads(keyData)
        keys = keyDataJson.keys()
        csvKeys = getCsvKeylist(keyDataJson, keys, [], "")
        print(csvKeys)
        
        # Get CSV datas
        csvData = []
        for competition in competitionsData:
            competitionValues = []
            
            #Get each data row for competition
            csvRow = getCsvDataRow(competition, keyDataJson, keys, [])
            csvData.append(csvRow)
        # print(csvData)

        # Open a new CSV file and write the header row
        with open('competitions.csv', 'w', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames = csvKeys)
            writer.writeheader()

        # Write data rows to the same CSV file
        with open('competitions.csv', 'a', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(csvData)

        print("Data has been written to competitions.csv")

    else:
        print('Did not get the json data')

except Exception as e:
    exc_type, exc_value, exc_traceback = sys.exc_info()
    print(f"Caught exception {exc_type} with message {e}")
    print("Traceback:")
    traceback.print_tb(exc_traceback)




['id', 'country', 'state', 'city', 'station', 'last_update', 'pollutant_id', 'pollutant_min', 'pollutant_max', 'pollutant_avg', 'pollutant_unit']
Caught exception <class 'PermissionError'> with message [Errno 13] Permission denied: 'competitions.csv'
Traceback:


  File "C:\Users\Manu\AppData\Local\Temp\ipykernel_8332\3814445019.py", line 36, in <module>
    with open('competitions.csv', 'w', encoding='utf-8', newline='') as file:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Manu\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 284, in _modified_open
    return io_open(file, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
